In [1]:
import numpy as np
import tensorflow as tf


print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

def normalize(array, std_dev, mean, scale):
    array = array/scale
    return array-mean/std_dev

folders = ["fold1","fold2","fold3","fold4","fold5"]

2.3.2


MODELS

In [2]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, InputLayer, RepeatVector, Bidirectional, TimeDistributed, Input
from tensorflow import keras
from tensorflow.keras import  regularizers
from tensorflow.keras.layers.experimental import RandomFourierFeatures

In [3]:
from tensorflow.keras.models import Model

input_shape=(32,16516)
input = Input(shape=input_shape)
L1 = LSTM(256,
                                     kernel_regularizer=regularizers.l2(0.00001),
                                         recurrent_regularizer=regularizers.l2(0.00001),
                                         bias_regularizer=regularizers.l2(0.00001),
          return_sequences=True,
                                     recurrent_dropout=0.3)(input)
L2 = LSTM(128,
                                     kernel_regularizer=regularizers.l2(0.00001),
                                         recurrent_regularizer=regularizers.l2(0.00001),
                                         bias_regularizer=regularizers.l2(0.00001),
                                     recurrent_dropout=0.3)(L1)
L3 = RepeatVector(input_shape[0])(L2)
L4 = LSTM(128,
                                     kernel_regularizer=regularizers.l2(0.00001),
                                         recurrent_regularizer=regularizers.l2(0.00001),
                                         bias_regularizer=regularizers.l2(0.00001),return_sequences=True,
                                     recurrent_dropout=0.3)(L3)
L5 = LSTM(256,
                                     kernel_regularizer=regularizers.l2(0.00001),
                                         recurrent_regularizer=regularizers.l2(0.00001),
                                         bias_regularizer=regularizers.l2(0.00001),return_sequences=True,
                                     recurrent_dropout=0.3)(L4)
output = TimeDistributed(Dense(16516))(L5)
model = Model(input, output)

In [4]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 16516)]       0         
_________________________________________________________________
lstm (LSTM)                  (None, 32, 256)           17175552  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               197120    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 32, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32, 128)           131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 32, 256)           394240    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 16516)        

In [5]:
dataset = np.load("dataset_overlap.npy")
classes = np.load("/home/andrea/Scrivania/MIVIA_ROAD_DB1/bin_classes.npy")

In [6]:
def unison_shuffled_copies(a, b):
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [7]:
def normalize(array, std_dev, mean, scale):
    array = array/scale
    return array-mean/std_dev

In [8]:
std_dev = np.std(dataset)
mean = np.mean(dataset)
maximum = np.max(dataset)
min = np.min(dataset)
scale = max(maximum, abs(min))
train_ds = normalize(dataset, std_dev, mean, scale)


In [9]:
filepath = "test.h5"
cb1 = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
)
cb2 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

cbs = [cb1,cb2]

In [9]:
#callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
    loss=tf.keras.losses.MAE,
    metrics=['accuracy']
)

In [10]:
EPOCHS = 100
history = model.fit(
    x= train_ds,
    y = train_ds,
    #validation_data=(val_ds, val_labels.reshape(-1,1)),
    batch_size=16,
    epochs=EPOCHS,
    validation_split=0.15
    #callbacks=cbs
)

Epoch 1/100
40/40 [==============================] - 13s 321ms/step - loss: 0.1022 - accuracy: 9.9206e-05 - val_loss: 0.1163 - val_accuracy: 0.0000e+00

In [11]:
X_pred = model.predict(train_ds)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])

ValueError: cannot reshape array of size 392155904 into shape (742,16516)